# Gender Classification from Text Blog

## Load dataset into notebook

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup
import string
import nltk
nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import re


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn import metrics

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hoangvo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/hoangvo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hoangvo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#!  conda install  openpyxl

In [3]:

#df = pd.read_excel('blog-gender-dataset/blog-gender-dataset.xlsx', engine='openpyxl')
headers = ["text", "class"]
text_df = pd.read_csv('blog-gender-dataset/blog-gender-dataset.csv')

In [4]:
text_df = text_df.iloc[:, [0, 1]]
text_df.columns = headers
text_df

,text,class
0,Guest Demo: Eric Iverson’s Itty Bitty Search\...,M
1,Who moved my Cheese??? The world has been de...,M
2,Yesterday I attended a biweekly meeting of an...,M
3,Liam is nothing like Natalie. Natalie never w...,F
4,"In the EU we have browser choice, but few know...",M
...,...,...
3226,It was a scavenger style race with checkpoints...,M
3227,Finally! I got a full day's work done. Almost ...,F
3228,"At the height of laughter, the universe is flu...",M
3229,"I like birds, especially woodpeckers and MOST ...",M


In [5]:
text_df.shape

(3231, 2)

In [6]:
text_df["class"].describe()

count     3226
unique       8
top          M
freq      1546
Name: class, dtype: object

In [7]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3231 entries, 0 to 3230
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3224 non-null   object
 1   class   3226 non-null   object
dtypes: object(2)
memory usage: 50.6+ KB


In [8]:
text_df[text_df['class']=="M"].shape

(1546, 2)

In [9]:
text_df[text_df['class']=="F"].shape

(1390, 2)

In [10]:
text_df = text_df.dropna()
text_df

,text,class
0,Guest Demo: Eric Iverson’s Itty Bitty Search\...,M
1,Who moved my Cheese??? The world has been de...,M
2,Yesterday I attended a biweekly meeting of an...,M
3,Liam is nothing like Natalie. Natalie never w...,F
4,"In the EU we have browser choice, but few know...",M
...,...,...
3226,It was a scavenger style race with checkpoints...,M
3227,Finally! I got a full day's work done. Almost ...,F
3228,"At the height of laughter, the universe is flu...",M
3229,"I like birds, especially woodpeckers and MOST ...",M


In [11]:
text_df["class"] = text_df["class"].replace(' F', 'F')
text_df["class"] = text_df["class"].replace('f', 'F')
text_df["class"] = text_df["class"].replace('F ', 'F')
text_df["class"] = text_df["class"].replace(' M', 'M')
text_df["class"] = text_df["class"].replace('m', 'M')
text_df["class"] = text_df["class"].replace(' M ', 'M')
text_df['class'].value_counts()

/Users/hoangvo/anaconda3/envs/Data-mining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/hoangvo/anaconda3/envs/Data-mining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hoangvo/anaconda3/envs/Data-mining/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

M    1677
F    1547
Name: class, dtype: int64

In [12]:
text_df1 = pd.read_csv('data/blogtext.csv')


In [13]:
text_df1 = text_df1[["text","gender"]]

In [14]:
text_df1.columns = headers
text_df1["class"] = text_df1["class"].replace('female', 'F')
text_df1["class"] = text_df1["class"].replace('male', 'M')
text_df1 = text_df1[text_df1["text"]!= ""]

In [15]:
#df_M = [text_df1[text_df1["class"]=="M"], text_df2[text_df2["class"]=="M"]]
df_M = [text_df1[text_df1["class"]=="M"]]
df_M = pd.concat(df_M)
df_M = df_M.sample(frac=1).reset_index(drop=True)
#df_F = [text_df1[text_df1["class"]=="F"], text_df2[text_df2["class"]=="F"]]
df_F = [text_df1[text_df1["class"]=="F"]]
df_F = pd.concat(df_F)
df_F = df_F.sample(frac=1).reset_index(drop=True)

In [16]:
#text_df = [text_df, df_M[:30000], df_F[:30000]]
text_df = [ df_M[:100000], df_F[:100000]]
text_df = pd.concat(text_df)
text_df = text_df.sample(frac=1).reset_index(drop=True)
text_df['class'].value_counts()

F    100000
M    100000
Name: class, dtype: int64

## Clean text

In [17]:
#import nltk
#nltk.download()

https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f

In [18]:
text_df["text"][1]

"                 I just realized that I haven't posted anything worthwhile in a LONG  TIME~  So I will try to post something.   Hm.I get my hair cut tomorrow. I am soooo nervous!!!! I hope they don't mess up or anything. But if they do its ok, because its just hair. It will grow!   Today was kinda a yucky day. It was just dull and I did absolutely nothign worthwile. I made 4 smoothies. Haha.  UGH!!! *tries to think of something worthwhile to post*  Well, I get my hair cut. I am quite nervous. Wait! I already posted about this. Haha.  Hmm..tonight I am going to try to go to this Sara thinng :)   I tyhink its raining outside which means we wont have a game today :( waaaa  I really wanted to play!  Yeah. this attempt is quite worthless..I betetr go get ready for tennis, bye!  Laura         "

In [19]:
#remove html
def remove_html(text):
    soup = BeautifulSoup(text, 'lxml')
    html_free = soup.get_text()
    return html_free

#remove punctuation
def remove_punctuation(text):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = " ".join([c for c in text if c not in punctuations])
    return no_punct
#textclean = text_df['text'].apply(lambda x : remove_punctuation(x))
#instantiate tokenizer
tokenizer = RegexpTokenizer(r'\w+')
#textclean = textclean.apply(lambda x)

In [20]:

def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df
data_clean = clean_text(text_df, 'text', 'text_clean')
data_clean.head()

,text,class,text_clean
0,Lazee. Lazy. Loused Up. I can't eve...,F,lazee lazy loused up i cant even be...
1,I just realized that I haven'...,F,i just realized that i havent...
2,i am updated my blog because&nbsp...,M,i am updated my blog becausenbsp ...
3,The guys at Rogers finally ...,M,the guys at rogers finally ...
4,Tapping is finished! yay! now...,F,tapping is finished yay now w...


In [21]:
#import nltk.corpus
#nltk.download('stopwords')
#from nltk.corpus import stopwords
stop = stopwords.words('english')
data_clean['text_clean'] = data_clean['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data_clean.head()

,text,class,text_clean
0,Lazee. Lazy. Loused Up. I can't eve...,F,lazee lazy loused cant even begin tell think m...
1,I just realized that I haven'...,F,realized havent posted anything worthwhile lon...
2,i am updated my blog because&nbsp...,M,updated blog becausenbsp siu kwok hanzi update...
3,The guys at Rogers finally ...,M,guys rogers finally figured solved slow intern...
4,Tapping is finished! yay! now...,F,tapping finished yay get boil laura aka cannibal


In [22]:
data_clean['text_clean'][0]

'lazee lazy loused cant even begin tell think missing tropfest somewhat something bothering fucking bashed trashed last nights housewarming party know travel oxford st sit super films well count urllink paradisco wasnt super mmwhat saying sound like know im talking came back really wasnt mood im also really bad feelings paranoia freaking like want time alone goddammit nurse fried braincells peace impatient reading things online impatient peoples lack consideration art man mean art want see beautiful breathless yet people would rather repeat crap pretending repeating hmm war comment dont think loz said schmidt mind howard stupid nazi incline agree lets talk party lot people came favorite guest loz fucking sweetheart really need talk thank much organise make comfortable house little time im lazy lazeee like example putting speedial phone fucked terminally stoned took many drugs last night first time life got terrified chems man didnt want sniff anything like get away dont know brain cope

In [23]:
#import nltk 
#nltk.download('punkt')
#from nltk.tokenize import sent_tokenize, word_tokenize

data_clean['text_tokens'] = data_clean['text_clean'].apply(lambda x: word_tokenize(x))

def word_stemmer(text):
    stem_text = [PorterStemmer().stem(i) for i in text]
    return stem_text

data_clean['text_tokens_stem'] = data_clean['text_tokens'].apply(lambda x: word_stemmer(x))
data_clean.head()

,text,class,text_clean,text_tokens,text_tokens_stem
0,Lazee. Lazy. Loused Up. I can't eve...,F,lazee lazy loused cant even begin tell think m...,"[lazee, lazy, loused, cant, even, begin, tell,...","[laze, lazi, lous, cant, even, begin, tell, th..."
1,I just realized that I haven'...,F,realized havent posted anything worthwhile lon...,"[realized, havent, posted, anything, worthwhil...","[realiz, havent, post, anyth, worthwhil, long,..."
2,i am updated my blog because&nbsp...,M,updated blog becausenbsp siu kwok hanzi update...,"[updated, blog, becausenbsp, siu, kwok, hanzi,...","[updat, blog, becausenbsp, siu, kwok, hanzi, u..."
3,The guys at Rogers finally ...,M,guys rogers finally figured solved slow intern...,"[guys, rogers, finally, figured, solved, slow,...","[guy, roger, final, figur, solv, slow, interne..."
4,Tapping is finished! yay! now...,F,tapping finished yay get boil laura aka cannibal,"[tapping, finished, yay, get, boil, laura, aka...","[tap, finish, yay, get, boil, laura, aka, cannib]"


In [24]:
#nltk.download('wordnet')
#from nltk.stem import WordNetLemmatizer

def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

data_clean['text_tokens_lemma'] = data_clean['text_tokens'].apply(lambda x: word_lemmatizer(x))
data_clean.head()

,text,class,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma
0,Lazee. Lazy. Loused Up. I can't eve...,F,lazee lazy loused cant even begin tell think m...,"[lazee, lazy, loused, cant, even, begin, tell,...","[laze, lazi, lous, cant, even, begin, tell, th...","[lazee, lazy, loused, cant, even, begin, tell,..."
1,I just realized that I haven'...,F,realized havent posted anything worthwhile lon...,"[realized, havent, posted, anything, worthwhil...","[realiz, havent, post, anyth, worthwhil, long,...","[realized, havent, posted, anything, worthwhil..."
2,i am updated my blog because&nbsp...,M,updated blog becausenbsp siu kwok hanzi update...,"[updated, blog, becausenbsp, siu, kwok, hanzi,...","[updat, blog, becausenbsp, siu, kwok, hanzi, u...","[updated, blog, becausenbsp, siu, kwok, hanzi,..."
3,The guys at Rogers finally ...,M,guys rogers finally figured solved slow intern...,"[guys, rogers, finally, figured, solved, slow,...","[guy, roger, final, figur, solv, slow, interne...","[guy, rogers, finally, figured, solved, slow, ..."
4,Tapping is finished! yay! now...,F,tapping finished yay get boil laura aka cannibal,"[tapping, finished, yay, get, boil, laura, aka...","[tap, finish, yay, get, boil, laura, aka, cannib]","[tapping, finished, yay, get, boil, laura, aka..."


In [25]:
#part of speech tagging and chunking
def word_pos_tagger(text):
    pos_tagged_text = nltk.pos_tag(text)
    return pos_tagged_text

nltk.download('averaged_perceptron_tagger')
data_clean['text_tokens_pos_tagged'] = data_clean['text_tokens'].apply(lambda x: word_pos_tagger(x))
data_clean.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hoangvo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,class,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma,text_tokens_pos_tagged
0,Lazee. Lazy. Loused Up. I can't eve...,F,lazee lazy loused cant even begin tell think m...,"[lazee, lazy, loused, cant, even, begin, tell,...","[laze, lazi, lous, cant, even, begin, tell, th...","[lazee, lazy, loused, cant, even, begin, tell,...","[(lazee, NN), (lazy, NN), (loused, VBD), (cant..."
1,I just realized that I haven'...,F,realized havent posted anything worthwhile lon...,"[realized, havent, posted, anything, worthwhil...","[realiz, havent, post, anyth, worthwhil, long,...","[realized, havent, posted, anything, worthwhil...","[(realized, VBN), (havent, NN), (posted, VBD),..."
2,i am updated my blog because&nbsp...,M,updated blog becausenbsp siu kwok hanzi update...,"[updated, blog, becausenbsp, siu, kwok, hanzi,...","[updat, blog, becausenbsp, siu, kwok, hanzi, u...","[updated, blog, becausenbsp, siu, kwok, hanzi,...","[(updated, VBN), (blog, NN), (becausenbsp, NN)..."
3,The guys at Rogers finally ...,M,guys rogers finally figured solved slow intern...,"[guys, rogers, finally, figured, solved, slow,...","[guy, roger, final, figur, solv, slow, interne...","[guy, rogers, finally, figured, solved, slow, ...","[(guys, NNS), (rogers, NNS), (finally, RB), (f..."
4,Tapping is finished! yay! now...,F,tapping finished yay get boil laura aka cannibal,"[tapping, finished, yay, get, boil, laura, aka...","[tap, finish, yay, get, boil, laura, aka, cannib]","[tapping, finished, yay, get, boil, laura, aka...","[(tapping, VBG), (finished, VBN), (yay, RB), (..."


In [26]:
# Remove short words less than 3
data_clean['fine'] = data_clean['text_tokens_lemma'].apply(lambda x: ' '.join([w for w in x if len(w)>3]))
# Count the length of characters
data_clean['clean_length'] = data_clean['fine'].apply(len)
# Remove rows where character length <= 5
data_clean = data_clean[data_clean.clean_length > 5]
data_clean.sample(n=15)


,text,class,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma,text_tokens_pos_tagged,fine,clean_length
114085,Lava lamps are awesome Even more so the...,M,lava lamps awesome even possum made wax oil u ...,"[lava, lamps, awesome, even, possum, made, wax...","[lava, lamp, awesom, even, possum, made, wax, ...","[lava, lamp, awesome, even, possum, made, wax,...","[(lava, NN), (lamps, VBZ), (awesome, JJ), (eve...",lava lamp awesome even possum made leave long ...,212
149143,Poet-seers who have achieved the hi...,M,poetseers achieved highest realisations endeav...,"[poetseers, achieved, highest, realisations, e...","[poetseer, achiev, highest, realis, endeavour,...","[poetseers, achieved, highest, realisation, en...","[(poetseers, NNS), (achieved, VBD), (highest, ...",poetseers achieved highest realisation endeavo...,332
196033,You're so gay. (filled with love.)...,F,youre gay filled love,"[youre, gay, filled, love]","[your, gay, fill, love]","[youre, gay, filled, love]","[(youre, NN), (gay, NN), (filled, VBN), (love,...",youre filled love,17
70772,THERE'S SOMETHING DEEPLY SURR...,M,theres something deeply surreal introducing bl...,"[theres, something, deeply, surreal, introduci...","[there, someth, deepli, surreal, introduc, blo...","[there, something, deeply, surreal, introducin...","[(theres, NNS), (something, NN), (deeply, RB),...",there something deeply surreal introducing blo...,480
156066,Have you ever had one of those days whe...,F,ever one days want throttle one coworkers one ...,"[ever, one, days, want, throttle, one, coworke...","[ever, one, day, want, throttl, one, cowork, o...","[ever, one, day, want, throttle, one, coworker...","[(ever, RB), (one, CD), (days, NNS), (want, VB...",ever want throttle coworkers today went work g...,323
181932,omg today i .... am gonna do everything...,F,omg today gonna everything dont wanna woohoo i...,"[omg, today, gon, na, everything, dont, wan, n...","[omg, today, gon, na, everyth, dont, wan, na, ...","[omg, today, gon, na, everything, dont, wan, n...","[(omg, NN), (today, NN), (gon, VB), (na, TO), ...",today everything dont woohoo damn excited last...,328
30928,so today was one of the best ...,F,today one best workouts ive well long time tha...,"[today, one, best, workouts, ive, well, long, ...","[today, one, best, workout, ive, well, long, t...","[today, one, best, workout, ive, well, long, t...","[(today, NN), (one, CD), (best, JJS), (workout...",today best workout well long time thats good m...,145
47022,hi dideley ho neighbour!,M,hi dideley ho neighbour,"[hi, dideley, ho, neighbour]","[hi, dideley, ho, neighbour]","[hi, dideley, ho, neighbour]","[(hi, NN), (dideley, NN), (ho, VBD), (neighbou...",dideley neighbour,17
23080,"Pursuant to the last post, urlLink Jus...",F,pursuant last post urllink desserts cream liqu...,"[pursuant, last, post, urllink, desserts, crea...","[pursuant, last, post, urllink, dessert, cream...","[pursuant, last, post, urllink, dessert, cream...","[(pursuant, NN), (last, JJ), (post, NN), (urll...",pursuant last post urllink dessert cream lique...,152
50642,As I stare into the depths of spa...,M,stare depths space worry stricken nook thought...,"[stare, depths, space, worry, stricken, nook, ...","[stare, depth, space, worri, stricken, nook, t...","[stare, depth, space, worry, stricken, nook, t...","[(stare, JJ), (depths, NNS), (space, NN), (wor...",stare depth space worry stricken nook thought ...,756


In [27]:
data_clean.shape

(197773, 9)

In [28]:
def preprocess_sample(text):
    #clean text
    newText = text.str.lower()
    newText = newText.apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    newText = newText.apply(lambda elem: re.sub(r"\d+", "", elem))
    #stopwords
    from nltk.corpus import stopwords
    stop = stopwords.words('english')
    newText = newText.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    #stemmer
    newText = newText.apply(lambda x: word_tokenize(x))
    newText_stem = newText.apply(lambda x: word_stemmer(x))
    #token lemma
    newText_lem = newText.apply(lambda x: word_lemmatizer(x))
    newText_tag = newText.apply(lambda x: word_pos_tagger(x))
 


## Declare X and Y data

https://pub.towardsai.net/does-a-machine-know-your-gender-based-on-your-tweets-43b14740fd54

In [29]:
# Import libraries
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import f1_score
# Bag-of-words features
bow_vectorizer = CountVectorizer(stop_words='english')
# Bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(data_clean['fine'])
df_bow = pd.DataFrame(bow.todense(), columns=bow_vectorizer.get_feature_names())
df_bow

,aaaa,aaaaa,aaaaaa,aaaaaaa,aaaaaaaa,aaaaaaaaa,aaaaaaaaaa,aaaaaaaaaaa,aaaaaaaaaaaa,aaaaaaaaaaaaa,...,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzas,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_clean['gender'] = data_clean['class'].apply(lambda x: 1 if x=='F' else 0)
# Splitting the data into training and test set
X = df_bow
y = data_clean['gender']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# Fitting on Logistic Regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
prediction_bow = logreg.predict_proba(X_test)
# Calculating the F1 score
# If prediction is greater than or equal to 0.5 than 1, else 0
# Gender, 0 = male and 1 = female
prediction_int = prediction_bow[:,1]>=0.5
prediction_int = prediction_int.astype(np.int)
# Calculating F1 score
log_bow = f1_score(y_test, prediction_int)
log_bow

In [ ]:
predictions = logreg.predict(X_test)

In [ ]:
predictions

In [ ]:
# Use score method to get accuracy of model
score = logreg.score(X_test, y_test)
print(score)

In [ ]:


cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [ ]:
predicted = clf.predict(X_test)
np.mean(predicted == y_test)

In [ ]:
#from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction.text import TfidfTransformer
#text_clf = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
#                      ('clf', MultinomialNB()),
 #                      ])
#text_clf = text_clf.fit(X_train, y_train)

In [ ]:
#predicted = text_clf.predict(X_test)
#np.mean(predicted == y_test)

In [ ]:
from sklearn.linear_model import SGDClassifier
clf_svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter_no_change=5, random_state=42)
_ = clf_svm.fit(X_train, y_train)
predicted_svm = clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

In [ ]:
X1 = text_df["text"]
y1 = text_df["class"].apply(lambda x: 1 if x=='F' else 0)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X1, y1, test_size=0.20)

https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('mnb', MultinomialNB(fit_prior=False)),
 ])
text_mnb_stemmed = text_mnb_stemmed.fit(X_train2, y_train2)
predicted_mnb_stemmed = text_mnb_stemmed.predict(X_test2)
np.mean(predicted_mnb_stemmed == y_test2)

Apply Neuron Network: https://realpython.com/python-keras-text-classification/